# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets   # for troubleshooting
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# From Dr. A:
import warnings
warnings.filterwarnings('ignore')


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
# Set file path
path = "output_data/cities.csv"

cities_df = pd.read_csv(path)

cities_df.head()

,City,Country,Lat,Long,Temp,Humidity,Clouds,Winds
0,Kodīnar,IN,20.7903,70.7031,76.86,66,0,8.55
1,Zagora,MA,30.3484,-5.8365,72.93,27,0,4.59
2,Kapaa,US,22.0752,-159.3190,82.38,77,20,4.00
3,Coquimbo,CL,-29.9533,-71.3436,62.33,59,0,12.66
4,Hobart,AU,-42.8794,147.3294,50.02,92,75,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Long"]]

# Fill NaN values and convert to float
ratings = cities_df["Humidity"].astype(float)

In [52]:
# Authenticate API key
gmaps.configure(api_key=g_key)

# Specify layout parameters
figure_layout = {'width': '800px', 'height': '420px', 'margin': '0 auto 0 auto'}

# Plot Heatmap
fig = gmaps.figure(map_type='HYBRID', layout = figure_layout, center=(34,-85), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations, weights=ratings,
    max_intensity=30, point_radius=3.0
)

# Add heat map layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:

drop_temp_lo = cities_df.index[cities_df['Temp'] < 72]
my_cities1 = cities_df.drop(drop_temp_lo, inplace=False)

drop_temp_hi = my_cities1.index[my_cities1['Temp'] > 80]
my_cities2 = my_cities1.drop(drop_temp_hi, inplace=False)

drop_humid_hi = my_cities2.index[my_cities2['Humidity'] > 40]
my_cities3 = my_cities2.drop(drop_humid_hi, inplace=False)

drop_cloudy = my_cities3.index[my_cities3['Clouds'] >= 50]
my_cities4 = my_cities3.drop(drop_cloudy, inplace=False)

drop_windy = my_cities4.index[my_cities4['Winds'] > 15]
my_cities = my_cities4.drop(drop_windy, inplace=False)

my_cities.reset_index(drop=True)

,City,Country,Lat,Long,Temp,Humidity,Clouds,Winds
0,Zagora,MA,30.3484,-5.8365,72.93,27,0,4.59
1,Vicuña,CL,-30.0319,-70.7081,78.76,28,0,7.23
2,San Rafael,AR,-34.6177,-68.3301,73.89,16,0,12.41
3,Oropesa,PE,-13.5928,-71.7719,73.26,28,40,11.50
4,Buraidah,SA,26.3260,43.9750,77.45,18,0,7.70
5,Ormara,PK,25.2088,64.6357,79.56,37,0,13.33
6,Kutum,SD,14.2000,24.6667,77.50,20,18,14.20
7,Sunrise Manor,US,36.2111,-115.0731,78.91,28,20,0.00
8,Jalu,LY,29.0331,21.5482,73.02,34,4,11.03
9,Abu Kamal,SY,34.4506,40.9171,72.86,29,48,4.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
# Store selected cities into new variable hotel_df
hotel_df = my_cities.copy()
hotel_df = hotel_df.reset_index(drop=True)

# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''

hotel_df


,City,Country,Lat,Long,Temp,Humidity,Clouds,Winds,Hotel Name
0,Zagora,MA,30.3484,-5.8365,72.93,27,0,4.59,
1,Vicuña,CL,-30.0319,-70.7081,78.76,28,0,7.23,
2,San Rafael,AR,-34.6177,-68.3301,73.89,16,0,12.41,
3,Oropesa,PE,-13.5928,-71.7719,73.26,28,40,11.50,
4,Buraidah,SA,26.3260,43.9750,77.45,18,0,7.70,
5,Ormara,PK,25.2088,64.6357,79.56,37,0,13.33,
6,Kutum,SD,14.2000,24.6667,77.50,20,18,14.20,
7,Sunrise Manor,US,36.2111,-115.0731,78.91,28,20,0.00,
8,Jalu,LY,29.0331,21.5482,73.02,34,4,11.03,
9,Abu Kamal,SY,34.4506,40.9171,72.86,29,48,4.21,


In [47]:
# Set fixed search parameters

target_search = 'Hotel'
target_radius = 5000
target_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():
    
    target_coordinates = f"{row['Lat']}, {row['Long']}"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    try:
        response = requests.get(base_url, params=params)
        nearest_hotel = response.json()['results'][0]['name']
    except:
        nearest_hotel = 'Nothing Found'
        
    pprint(response.json())
    print(nearest_hotel)
    
    hotel_df.loc[index, 'Hotel Name'] = nearest_hotel

hotel_df

{'html_attributions': [],
 'next_page_token': 'Aap_uEAaqKyXGXkk6RltgHjWsgDACebdpF9dKtnJADFBLVnhIir7tp7KokxkH3abHnJgDPbRMk6MVW88vrcZhEHp8EbWUE8D9ObKkfTLCB-q4bOAJnVbsGfulT4_m7mE__Dfj4ypxeigb6gbt9bK52Xvrjx6dcDJzSGlHYuqrkSr65a4CDIMqs-3XpLT_Rx41vdQeQJqazXcJ4xFfxF4MyUfB1_hbwalziw2AbFScmoyJnH-LsVSBZqCRCWfeXsRMpYY3R0w0Hni7EEj-YA94MvvH_byyikmThZOCwSY4pmgR73hKBCN2jbfkRoixvfpe4R3B8lc4gwEoFebuoVMsfOii7yELoNsRogwaf0WDuAPralkKtbff8ajbygEC7tL6skntZlAH5zrYgscepCrtm-iTsbwUd5dHhA-_lhUNk47eTVo59w86h6R4qLZrmg9tQo',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 30.3336957, 'lng': -5.8365376},
                           'viewport': {'northeast': {'lat': 30.33509537989272,
                                                      'lng': -5.835545620107278},
                                        'southwest': {'lat': 30.33239572010728,
                                                      'lng': -5.838245279892722}}},
              'icon': 'https://maps.gstatic.co

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -30.033739, 'lng': -70.708452},
                           'viewport': {'northeast': {'lat': -30.03242757010728,
                                                      'lng': -70.70710667010728},
                                        'southwest': {'lat': -30.03512722989272,
                                                      'lng': -70.70980632989271}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Solar Madariaga Hotel',
              'opening_hours': {'open_now': True},
              'place_id': 'ChIJr1KCVSwakJYRvsCTKQIciXE',
              'plus_code': {'compound_code': 'X78R+GJ Vicuña, Chile',
                            'global_cod

{'html_attributions': [],
 'next_page_token': 'Aap_uED2UzC6lPToaROYvDl8EgrItjE0QNlbSbJB_g-fV-n92IOpo1E3IvC60eL0_vu5yfBBtlFYuK4affOk90qSatLbtpPrVO8VRjGgJ1sKwfBxh5HwrTrLzgu26qz6l8a-BGsVLiZ9gVW0uaouuVfiHwIjOY3qvRQBT2yKichiuMmDs2q8tENmzWTrAqSnVKApujaTOTWGJVxQ6X-vO-Rl3wVlNd-V8iEWlpqUD8SX985EsHNtjAtf_WBLVIqPqxQz2fQs4oyIyUa40yrKbx1NYVReCp3-cZpwnB-Ye76DihjN9uDm1HbMLB1e0yhPmmsBI_mkTs3LOWYB0uDlkKvDF7tq7_sNYmpK7rc3rv-wD3mFqxr_2YNxX-xfK-LDBjx2j0oNf0-6pY9aODY5H8AgVt1Z2jCQTo2cNI_G7C9aE23CCEK2BCesy-ywZb4O8ok',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -34.6130715, 'lng': -68.3538629},
                           'viewport': {'northeast': {'lat': -34.61175407010728,
                                                      'lng': -68.35247182010727},
                                        'southwest': {'lat': -34.61445372989272,
                                                      'lng': -68.35517147989272}}},
              'icon': 'https://maps.gstati

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Nothing Found
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 26.3631102, 'lng': 43.9302354},
                           'viewport': {'northeast': {'lat': 26.36441187989272,
                                                      'lng': 43.93168727989273},
                                        'southwest': {'lat': 26.36171222010728,
                                                      'lng': 43.92898762010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Best Western Plus Buraidah',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 720,
                          'html_attribution

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.2492975, 'lng': 64.5966851},
                           'viewport': {'northeast': {'lat': 25.25054962989272,
                                                      'lng': 64.59796382989273},
                                        'southwest': {'lat': 25.24784997010727,
                                                      'lng': 64.59526417010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Alqadri hotel zero point ormara',
              'photos': [{'height': 2576,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/101123423469979260886">

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Nothing Found
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 36.238521, 'lng': -115.118824},
                           'viewport': {'northeast': {'lat': 36.24022672989272,
                                                      'lng': -115.1150825},
                                        'southwest': {'lat': 36.23752707010727,
                                                      'lng': -115.1231017}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Cannery Casino & Hotel',
              'photos': [{'height': 480,
                          'html_attributions': ['<a '
                                                'href=

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 29.0398126, 'lng': 21.5353298},
                           'viewport': {'northeast': {'lat': 29.04116697989273,
                                                      'lng': 21.53658027989272},
                                        'southwest': {'lat': 29.03846732010728,
                                                      'lng': 21.53388062010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Thuraya Tourist Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 405,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.

,City,Country,Lat,Long,Temp,Humidity,Clouds,Winds,Hotel Name
0,Zagora,MA,30.3484,-5.8365,72.93,27,0,4.59,Hôtel Zagora Riad Salam
1,Vicuña,CL,-30.0319,-70.7081,78.76,28,0,7.23,Solar Madariaga Hotel
2,San Rafael,AR,-34.6177,-68.3301,73.89,16,0,12.41,Tierra Mora
3,Oropesa,PE,-13.5928,-71.7719,73.26,28,40,11.50,Nothing Found
4,Buraidah,SA,26.3260,43.9750,77.45,18,0,7.70,Best Western Plus Buraidah
5,Ormara,PK,25.2088,64.6357,79.56,37,0,13.33,Alqadri hotel zero point ormara
6,Kutum,SD,14.2000,24.6667,77.50,20,18,14.20,Nothing Found
7,Sunrise Manor,US,36.2111,-115.0731,78.91,28,20,0.00,Cannery Casino & Hotel
8,Jalu,LY,29.0331,21.5482,73.02,34,4,11.03,Thuraya Tourist Hotel
9,Abu Kamal,SY,34.4506,40.9171,72.86,29,48,4.21,Nothing Found


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [49]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, hotel_info)

# Display figure

fig = gmaps.figure()
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))